#	Pré-requis

##	Python 3.7
L’ensemble des scripts de l’outils sont écrits en langage Python. La version utilisée est Python 3.7.

#Module Trafficintelligence
Afin d’utiliser l’outil de simulation, il est nécessaire d’installer le module ‘Trafficintelligence’ développé par le professeur Nicolas Saunier et ses collaborateurs. 
Disponible à l’adresse suivante : https://bitbucket.org/Nicolas/trafficintelligence/.
Ce package est également installable via PIP
Pour plus d’informations, se référer à la documentation en ligne.

##	Modules annexes
Certains des modules suivants sont préalablement installés avec Python 3.7, d’autres nécessitent une installation de votre part.
-	Copy
-	Itertools
-	Maths
-	Matplotlib
-	Networkx
-	Numpy
-	Scipy
Tous ces modules sont disponibles via PIP.

# Création d’un monde
##	Alignements
Un alignement est un objet de type moving.Trajectory. Par exemple, si vous souhaitez créer un alignement droit d'une largeur de 3.7m, partant du point A(0,0) allant au point B(0,500), suivre les étapes suivantes :

In [3]:
from trafficintelligence import moving 
import network

A = moving.Point(0,0)
B = moving.Point(0,500)
alignmentPoints = moving.Trajectory.fromPointList([A,B])

alignment = network.Alignment(idx=0, width=3.7, points=alignmentPoints)

## Distributions
Une distribution est un objet de type network.Distribution. Il est utile lors de la création de générateur d'utilisateurs. Une distribution permet de représenter une distribution par ses paramètres(moyenne, écart-type, et constante dans le cas d'une distribution dégénérée, courbe de fréquences cumulées, nom, et type de la distribution. 
Ainsi, chaque variable soumise à un tirage aléatoire prend ses paramètres à partir de cet objet Distribution.
Pour un usager, il est nécessaire d'instancier les paramètres tau, dn, headway, length, speed:

In [ ]:
distributions = {'dn': network.Distribution(cdf=None,
                                            degeneratedConstant=None,
                                            distributionName='norm', 
                                            distributionType='theoric', 
                                            loc=8.333333,
                                            scale=1),
                 'headway':network.Distribution(cdf=None,
                                                degeneratedConstant=None,
                                                distributionName='expon', 
                                                distributionType='theoric', 
                                                loc=1.5,
                                                scale=2.1),
                 'length':network.Distribution(cdf=None,
                                               degeneratedConstant=None,
                                               distributionName='norm',
                                               distributionType='theoric',
                                               loc=7,
                                               scale=1),
                 'speed':network.Distribution(cdf=None,
                                                degeneratedConstant=None,
                                                distributionName='norm', 
                                                distributionType='theoric', 
                                                loc=14,
                                                scale=3),
                 'tau':network.Distribution(cdf=None,
                                                degeneratedConstant=None,
                                                distributionName='norm', 
                                                distributionType='theoric', 
                                                loc=2,
                                                scale=.5)}


## Générateurs d'utilisateurs
Un générateur d'utilisaterus est un object de type network.UserInput. Il s'agit d'un point d'entrée sur le réseau qui attribue aux usagers produits des caractéristiques(longueur du véhicule, dn, t, temps inter-véhiculaire, vitesse désirée)  è partir de distributions aléatoires propres à chaque point d'entrée.
Afin de créer un générateur d'usagers relié à l'alignement précédant, suivre les étapes suivantes:

In [ ]:
userInput = network.UserInput(idx=0, alignmentIdx=alignment.idx,
                          distributions=distributions)

## Dispositifs de contrôle
Un dispositif de contrôle est un objet de type network.ControlDevice. Cette structure de donnée permet de prendre en compte des signalisation d'arrêt, de cédez le passage, ainsi que des feux tricolores. Un dispositif de contrôle est forcément lié à un alignement. On crée ici un panneau d'arrêt.

In [ ]:
controlDevice = network.ControlDevice(idx = 0, 
                           initialState = 'stop',
                           alignmentIdx=0,
                           greenTime=0,
                           redTime=float('inf'), 
                           category=1)  

## Monde
Le 'monde' est la structure de données qui rassemble les alignements, dispositifs de contrôle, générateurs d'usagers, et les usagers (la création de ceux-ci est traitée par la suite). Il est nécessaire d'ajouter ces attributs à l'objet network.World crée, puis de lier l'objet World à un graphe.

In [ ]:
# création de l'objet World
world = network.World(alignments = [alignment])#, 
                       #controlDevices = [cd], 
                       #userInputs = [userInput])
# corresponsdance des extremites de l'alignement avec les futures noeuds du graphe
world.getAlignmentById(alignment.idx).graphCorrespondance = [0,1]
# liaison du graphe au monde
world.getGraph()

Il a été crée un monde où il sera possible de générer des usagers qui circuleront. Par la suite, il est possible de asuvegarder le monde crée puis de le charger à partir des méthodes save et load de la classe 'World'.
![alt text](monde.png "monde dessin")





## Paramètres de simulation

Le fichier 'config.yml' rassemble l'ensemble des paramètres de configuration nécessaires pour la réalisation d'une simulation:
- duration: durée de la simulation (s)
- interactionDistance: distance minimale d'interaction (m)
- minimumTimeHeadway: temps inter-véhiculaire minimal (s)
- timeStep: pas de temps (s)
- seed: graine

# Simulation de trafic routier sur un lien

Pour rappel : un lien est représenté par une succession d'alignements. Afin de simuler du trafic routier sur un lien, il est nécessaire qu'un userInput soit lié à l'alignement d'entrée des véhicules.

In [ ]:
import network
import simulation

world = network.World.load('simple-net.yml')
sim = simulation.Simulation.load('config.yml')   
sim.run(world)
world.plotUserTrajectories(sim.timeStep)

# Analyse des données produites

À venir